In [1]:
import pandas as pd
import json
import numpy as np

In [2]:
##### QUESTION 2 #####

def open_file_mle_output(file):
    output_lst = []
    tokens_tags = []
    with open(file, encoding = 'utf-8') as curr:
        for line in curr.readlines():
            if len(line.strip()) != 0:
                tokens_tags.append(line.strip().split("\t"))
                 # print(line.strip().split("\t"))
    for token_tag in tokens_tags:
        output_lst.append(token_tag)
    return output_lst
                
twitter_train_lst = open_file_mle_output("twitter_train.txt")
print(twitter_train_lst)

def get_unique_words(lst):
    # To get the number of unique words
    train_word_lst = []

    for pair in lst:
        if pair[0] not in lst:
            train_word_lst.append(pair[0])
    count = len(train_word_lst)
    return count

[['RT', '~'], ['@USER_44285fcc', '@'], [':', '~'], ['I', 'O'], ['see', 'V'], ['u', 'O'], ['Bron', '^'], ['...', ','], ['31', '$'], ['right', 'R'], ['quick', 'A'], ['@USER_a887f5eb', '@'], ['I', 'O'], ['am', 'V'], ['secure', 'A'], ['.', ','], ['@USER_1d9aecf2', '@'], ['hahaaha', '!'], ['!!', ','], ['i', 'O'], ['dont', 'V'], ['care', 'V'], ['if', 'P'], ['she', 'O'], ['realises', 'V'], ['i', 'O'], ['will', 'V'], ['be', 'V'], ['like', 'P'], ['shut', 'V'], ['up', 'T'], ['teacher', 'N'], ['and', '&'], ['go', 'V'], ['do', 'V'], ['your', 'D'], ['work', 'N'], ['!!', ','], ['hehe', '!'], ['then', 'R'], ['i', 'O'], ['will', 'V'], ['be', 'V'], ['in', 'P'], ['trouble', 'N'], ['!!', ','], ['Getting', 'V'], ['insulation', 'N'], ['put', 'V'], ['in', 'P'], ['attic', 'N'], ['today', 'N'], ['.', ','], ['They', 'O'], ['was', 'V'], ['sposed', 'A'], ['2', 'P'], ['be', 'V'], ['here', 'R'], ['at', 'P'], ['7:30', '$'], ['am', 'N'], [',', ','], ['but', '&'], ['got', 'V'], ['here', 'R'], ['at', 'P'], ['8:30', '$

In [3]:
##### QUESTION 2 #####
def get_tag_count(twitter_train_lst):
    # Contains the count of all the tags in the twitter_train
    tag_count_dic = {}

    for pair in twitter_train_lst:
        if pair[1] not in tag_count_dic.keys():
            tag_count_dic[pair[1]] = 1
        else:
            tag_count_dic[pair[1]] += 1
    return tag_count_dic

def get_token_tag_count(twitter_train_lst):
    # Contains the count of each token to a respective tag in twitter_train
    token_tag_count_dic = {}

    for pair in twitter_train_lst:
        if pair[0] not in token_tag_count_dic.keys():
            token_tag_count_dic[pair[0]] = {}
            token_tag_count_dic[pair[0]][pair[1]] = 1
        else:
            if pair[1] not in token_tag_count_dic[pair[0]].keys():
                token_tag_count_dic[pair[0]][pair[1]] = 1
            else:
                token_tag_count_dic[pair[0]][pair[1]] += 1
    return token_tag_count_dic
    #print(token_tag_count_dic)

    
def dic_to_file(dic, file):
    updated_file = open(file, "w", encoding = "utf-8")
    for key, value in dic.items():
        updated_file.write(key + " " + value + "\n")
    updated_file.close()

In [4]:
def mle_output(train_file):
    sigma = 1
    output_prob_dic = {}
    
    # Get the twitter_train_lst out from the train_file
    twitter_train_lst = open_file_mle_output(train_file)
    #print(twitter_train_lst)
    
    # Count the number of unique words
    unique_words = get_unique_words(twitter_train_lst)
    #print(unique_words)
    
    # Dict of tags and their respective counts
    tag_counts_dic = get_tag_count(twitter_train_lst)
    #print(tag_counts_dic)
    
    # Dict of Dict for tokens and their respective tag counts
    token_tag_counts_dic = get_token_tag_count(twitter_train_lst)
    
    for token, tag_dic in token_tag_counts_dic.items():
        for tag, count in tag_dic.items():
            prob = (count + sigma)/(tag_counts_dic[tag] + sigma * (unique_words + 1))
            output_prob_dic[token + " " + tag] = str(prob)
    return output_prob_dic
    
mle_output_dic = mle_output('twitter_train.txt')
#print(mle_output_dic)

# Update the naive_output_probs txt file
dic_to_file(mle_output_dic, 'naive_output_probs.txt')

In [5]:
#### Question 2b #####
# function to read in the in_output_probs_file
def open_file_into_dic(file):
    output_dic = {}
    token_tags = []
    output_probs_lst = []
    with open(file, encoding = 'utf-8') as curr:
        for line in curr.readlines():
            token, tag, prob = line.split(" ")
            
            if token not in output_dic.keys():
                output_dic[token] = {}
                output_dic[token][tag] = prob.replace("\n", "")
            else:
                output_dic[token][tag] = prob.replace("\n", "")
    return output_dic

def open_file_into_lst(file):
    output_lst = []
    with open(file, encoding = 'utf-8') as curr:
        for line in curr.readlines():
            if len(line.strip()) != 0:
                output_lst.append(line.replace("\n", ""))
    return output_lst

def get_max_key(dic):
    keys_lst = list(dic.keys())
    values_lst = list(dic.values())
    return keys_lst[values_lst.index(max(values_lst))]


def write_lst_to_file(lst, file):
    update_file = open(file, "w", encoding = "utf-8")
    for item in lst:
        update_file.write(item + "\n")
    update_file.close()
    
        


In [6]:
def naive_predict(in_output_probs_filename, in_test_filename, out_prediction_filename):
    output_lst = []
    # Read in the tokens from in_output_probs_filename into a dictionary
    token_tag_prob_dic = open_file_into_dic(in_output_probs_filename)
    print(token_tag_prob_dic)
    
    # Read in the tokens from in_test_filename into a list
    test_tokens_lst = open_file_into_lst(in_test_filename)
    
    # For each test_token, check if it is present in the token_tag_prob_dic
    # IF PRESENT: get the highest probability tag to it
    # ELSE: Assign tag 'Y' which is the lowest likelihood tag since unseen token means it 
    #       a rare occurrence hence should be a low appearance tag
    for token in test_tokens_lst:
        if token in token_tag_prob_dic.keys():
            max_tag = get_max_key(token_tag_prob_dic[token])
            output_lst.append(max_tag)
        else:
            if token[0:5] == "@USER":
                output_lst.append("@")
            elif token[0:4] == "http":
                output_lst.append("U")
            elif token[0] == "#":
                output_lst.append("#")
            else:
                output_lst.append("Y")
            
    # write the output_lst to the out_prediction_file
    write_lst_to_file(output_lst, out_prediction_filename)


naive_predict("naive_output_probs.txt", "twitter_dev_no_tag.txt", "naive_predictions.txt")

{'RT': {'~': '0.013426156141223273', 'N': '0.00011292417141889222', 'V': '0.00039003733214464814'}, '@USER_44285fcc': {'@': '0.00012227181023415052'}, ':': {'~': '0.011685728493286921', ',': '0.005910029837043838', 'E': '0.00044455734789787885', 'G': '0.0001905608842025027'}, 'I': {'O': '0.015995189416716776', '$': '0.00012655024044545685', 'E': '0.00012701638511367967'}, 'see': {'V': '0.0012258316153117513'}, 'u': {'O': '0.003427540589296452'}, 'Bron': {'^': '0.00012080212611741966'}, '...': {',': '0.005508377323846684', '~': '0.003853804077573347'}, '31': {'$': '0.00018982536066818528'}, 'right': {'R': '0.0007977417771232204', '!': '0.00018753516284303307', 'A': '0.0004886988393402566', 'N': '0.00011292417141889222'}, 'quick': {'A': '0.00012217470983506414'}, '@USER_a887f5eb': {'@': '0.00012227181023415052'}, 'am': {'V': '0.0007243550454114893', 'N': '0.0001693862571283383'}, 'secure': {'A': '0.00012217470983506414', 'V': '0.00011143923775561376'}, '.': {',': '0.02685333945375258', '

In [7]:
##### QUESTION 3
# 3(a) The probability on the RHS can be calculated by applying Bayes' Rule.
#      P(y = j | x = w) = P(x = w | y = j) * (P(y = j)/P(x=w))

In [8]:
# Take in the training file and output a dict containing the count for all tokens
def calc_token_count(file):
    twitter_train_lst = open_file_mle_output(file)
    output_dic = {}
    for pair in twitter_train_lst:
        if pair[0] not in output_dic.keys():
            output_dic[pair[0]] = 1
        else:
            output_dic[pair[0]] += 1
    return output_dic

In [9]:
# Takes in the token_tag_prob_dict and adjusts each of the probability, outputs a new dict
def adjust_prob_naive2(train_file, tokentag_prob_dic):
    sigma = 1
    
    output_dic = {}
    
    # Get the twitter_train_lst out from the train_file
    twitter_train_lst = open_file_mle_output(train_file)
    
    # Count the number of unique words
    unique_words = get_unique_words(twitter_train_lst)
    
    # Dict of tags and their respective counts
    tag_counts_dic = get_tag_count(twitter_train_lst)
    
    # Apply function to calculate the total count of each word
    token_count_dic = calc_token_count(train_file)
    
    for token in tokentag_prob_dic.keys():
        for tag, curr_prob in tokentag_prob_dic[token].items():
            updated_prob = float(curr_prob) * ((tag_counts_dic[tag] + sigma*(unique_words+1)) 
                                               /(token_count_dic[token] + sigma*(unique_words+1)))
            if token not in output_dic.keys():
                output_dic[token] = {}
                output_dic[token][tag] = updated_prob
            else:
                output_dic[token][tag] = updated_prob
    return output_dic

    

In [10]:
def naive_predict2(in_output_probs_filename, in_train_filename, in_test_filename, out_prediction_filename):
    output_lst = []
    # Read in the tokens from in_output_probs_filename into a dictionary
    token_tag_prob_dic = open_file_into_dic(in_output_probs_filename)
    
    # Read in the tokens from in_test_filename into a list
    test_tokens_lst = open_file_into_lst(in_test_filename)
    
    # Adjust the output_probabilities by multiplying by the [count(y = j) + sigma(num_words + 1)]
    naive2_token_tag_output_dic = adjust_prob_naive2(in_train_filename, token_tag_prob_dic)
    
    # For each test_token, check if it is present in the token_tag_prob_dic
    # IF PRESENT: get the highest probability tag to it
    # ELSE: Assign tag 'V'
    for token in test_tokens_lst:
        if token in naive2_token_tag_output_dic.keys():
            max_tag = get_max_key(naive2_token_tag_output_dic[token])
            output_lst.append(max_tag)
        else:
            # using information from twitter_train.txt to predict tags for certain tokens
            if token[0:5] == "@USER":
                output_lst.append("@")
            elif token[0:4] == "http":
                output_lst.append("U")
            elif token[0] == "#":
                output_lst.append("#")
            else:
                output_lst.append("V")
            
    # write the output_lst to the out_prediction_file
    write_lst_to_file(output_lst, out_prediction_filename)
    
naive_predict2("naive_output_probs.txt", "twitter_train.txt", "twitter_dev_no_tag.txt", "naive_predictions2.txt")

In [11]:
##### QUESTION 4 #####

In [12]:
# helper method to write dic to file
def write_dic_to_file(dic, file):
    update_file = open(file, "w", encoding = "utf-8")
    for tag_ij, prob in dic.items():
        update_file.write(tag_ij + " : " + prob + "\n")
    update_file.close()


In [13]:
# read_file method that returns a list containing 
def read_file_for_trans(file):
    output_lst = [[]] # list of list, inner list is for each tweet
    index = 0
    with open(file, encoding = 'utf-8') as curr:
        for line in curr.readlines():
            if len(line.strip()) != 0:
                token, sep, tag = line.strip().partition('\t')
                output_lst[index].append(tag)
            else:
                output_lst.append([])
                index += 1
    output_lst.pop()
    # For every set of tweet_tags, we need to include START and STOP tags
    for tweet_tags in output_lst:
        tweet_tags.insert(0,'START')
        tweet_tags.append('STOP')
    return output_lst
    
train_tag_seq_lst = read_file_for_trans('twitter_train.txt')
print(train_tag_seq_lst)

[['START', '~', '@', '~', 'O', 'V', 'O', '^', ',', '$', 'R', 'A', 'STOP'], ['START', '@', 'O', 'V', 'A', ',', 'STOP'], ['START', '@', '!', ',', 'O', 'V', 'V', 'P', 'O', 'V', 'O', 'V', 'V', 'P', 'V', 'T', 'N', '&', 'V', 'V', 'D', 'N', ',', '!', 'R', 'O', 'V', 'V', 'P', 'N', ',', 'STOP'], ['START', 'V', 'N', 'V', 'P', 'N', 'N', ',', 'O', 'V', 'A', 'P', 'V', 'R', 'P', '$', 'N', ',', '&', 'V', 'R', 'P', '$', ',', 'R', 'O', 'V', 'D', 'P', ',', 'STOP'], ['START', '^', '^', 'V', 'P', '^', '^', 'P', 'D', '$', 'N', 'N', ',', '^', '^', 'P', 'D', '$', 'N', 'N', 'STOP'], ['START', '#', '#', '$', ',', 'V', 'R', ',', 'A', 'N', ',', 'V', 'R', ',', 'A', 'N', ',', '^', '^', ',', 'N', ',', 'G', 'N', ',', 'N', '~', 'U', 'STOP'], ['START', 'L', 'N', ',', 'STOP'], ['START', '!', ',', 'P', 'D', 'D', 'V', 'A', ',', 'D', 'N', 'V', 'D', 'N', 'P', 'D', 'N', ',', 'R', 'V', 'O', 'P', ',', 'V', 'V', 'R', '&', 'R', 'R', 'V', 'O', 'V', '!', 'STOP'], ['START', '@', 'O', 'V', 'O', ',', 'L', 'R', 'R', 'D', '^', 'N', ',

In [27]:
# Obtain a dictionary where Key: Tag(i), Value: Dictionary(Key: Tag(j), Value: Count)
def get_trans_prob(tag_lst):
    output_dic = {}
    tag_count_dic = {}
    # to produce the dic of tag_i -> tag_j
    for tweet_tag_lst in tag_lst:
        #print(tweet_tag_lst)
        total_tags = len(tweet_tag_lst)
        for i in range(0, total_tags):
            if i == total_tags - 1:
                continue
            elif tweet_tag_lst[i] not in output_dic.keys():
                output_dic[tweet_tag_lst[i]] = {}
                output_dic[tweet_tag_lst[i]][tweet_tag_lst[i+1]] = 1
            else:
                if tweet_tag_lst[i+1] not in output_dic[tweet_tag_lst[i]].keys():
                    output_dic[tweet_tag_lst[i]][tweet_tag_lst[i+1]] = 1
                else:
                    output_dic[tweet_tag_lst[i]][tweet_tag_lst[i+1]] += 1
    # to produce the tag_count dict
    for tag_i in output_dic.keys():
        for tag_j in output_dic[tag_i].keys():
            if tag_i not in tag_count_dic:
                tag_count_dic[tag_i] = output_dic[tag_i][tag_j]
            else:
                tag_count_dic[tag_i] += output_dic[tag_i][tag_j]
    
    for tag in list(tag_count_dic.keys()):
        if tag != 'START' and tag not in output_dic['START'].keys():
            output_dic['START'][tag]= 0
            
    return output_dic, tag_count_dic

trans_count_dic, tag_count_dic = get_trans_prob(train_tag_seq_lst)
# print(trans_count_dic)
# print(tag_count_dic)

#To get the tag with the highest probability of appearing
def max_prob_tag(file):
    output_lst = [[]] # list of list, inner list is for each tweet
    index = 0
    with open(file, encoding = 'utf-8') as curr:
        for line in curr.readlines():
            if len(line.strip()) != 0:
                token, sep, tag = line.strip().partition('\t')
                output_lst[index].append(tag)
            else:
                output_lst.append([])
                index += 1
    output_lst.pop()

    total_tag_count = 0
    v_count = 0
    for tags in output_lst:
        total_tag_count += len(tags)
        for tag in tags:
            if tag =='V':
                v_count += 1
    return v_count/total_tag_count


0.15171351559491722

In [42]:
# Function to calculate the count(tag(i) -> tag(j))
def calc_trans_prob(dic, dic2):
    sigma = 0.01
    output_dic = {}
    unique_tags = len(dic2)
    tag_lst = list(dic2.keys())
    for tag_i in dic.keys():
        for tag_j in dic[tag_i].keys():
            tag_i_j_count = dic[tag_i][tag_j]
            # print(tag_i_j_count)
            #output_dic[tag_i + " , " + tag_j] = tag_i_j_count
            output_dic[tag_i + " " + tag_j] = str((tag_i_j_count + sigma)/(dic2[tag_i] + sigma *(unique_tags + 1)))
    return output_dic

trans_prob_dic = calc_trans_prob(trans_count_dic, tag_count_dic)
print(trans_prob_dic)

{'START ~': '0.1434798005938598', 'START @': '0.30329528635121267', 'START V': '0.07900878077129134', 'START ^': '0.06629618531332009', 'START #': '0.01816993107957177', 'START L': '0.03633078173381641', 'START !': '0.05903184505162223', 'START N': '0.05721575998619775', 'START ,': '0.01453776094872284', 'START D': '0.029974484004830784', 'START O': '0.0726524830423057', 'START A': '0.03542273920110418', 'START G': '0.00727342068702498', 'START S': '0.002733208023463819', 'START P': '0.019077973612284003', 'START E': '0.008181463219737213', 'START R': '0.030882526537543017', 'START $': '0.00727342068702498', 'START X': '0.0018251654907515866', 'START U': '0.002733208023463819', 'START &': '0.002733208023463819', 'START M': '0.0009171229580393546', 'START Z': '0.0009171229580393546', 'START T': '9.080425327122322e-06', 'START Y': '9.080425327122322e-06', '~ @': '0.47303421932827994', '~ O': '0.055435707641459026', '~ U': '0.07522710629960219', '~ ~': '0.03366516911750154', '~ #': '0.021

In [43]:
# Getting output probability using previously define MLE function
mle_output_dic_q4 = mle_output('twitter_train.txt')

# Update the naive_output_probs txt file
dic_to_file(mle_output_dic_q4, 'output_probs.txt')

# Calculating transition probability
def mle_trans(train_file, output_file):
    output_dic = {}
    # Take in the training file and convert it into a list of list
    # Inner List: Sequence of tags for each tweet in train file
    train_tag_seq_lst = read_file_for_trans(train_file)
    
    # Use the list to then generate a dict of dict, Outer dict: Tag
    # Inner dict: Count of subsequent tags
    trans_count_dic, tag_count_dic = get_trans_prob(train_tag_seq_lst)
    
    # Calculate the transition probabilities and insert into a dict
    trans_prob_dic = calc_trans_prob(trans_count_dic, tag_count_dic)
    
    # Write the trans_probs into trans_probs.txt
    write_dic_to_file(trans_prob_dic, output_file)

mle_trans("twitter_train.txt", "trans_probs.txt")


In [17]:
# Function to read file into the list
def file_to_lst(file):
    output_lst = [[]]
    index = 0
    with open(file, encoding = 'utf-8') as curr:
        for line in curr.readlines():
            if len(line.strip()) != 0:
                output_lst[index].append(line.replace("\n", ""))
            else:
                output_lst.append([])
                index += 1
    output_lst.pop()
    return output_lst
                
tweets_no_tag_lst = file_to_lst("twitter_dev_no_tag.txt")
print(tweets_no_tag_lst)

[['@USER_0240cb3a', 'Yeah', 'Im', 'goodie', 'just', 'in', 'the', 'lab', 'working', '...', 'Wow', '!', 'I', 'need', 'to', 'be', 'where', 'you', 'at', '!', 'lol', 'let', 'me', 'visit'], ['Does', 'it', 'feel', 'like', 'Summer', 'to', 'you', '?', 'It', 'does', 'in', 'Myrtle', 'Beach', ',', 'South', 'Carolina', 'its', '85', 'degrees'], ['@USER_341450c3', 'no', 'one', 'cares', '.', 'ppl', 'are', 'used', 'to', 'foolish', 'grabs', 'for', 'attention', '.', "they'll", 'just', '*', 'shrug', '*'], ['Red', 'wine', 'in', 'beautiful', 'china', '....'], ['RT', '@USER_f5eeff1e', ':', 'RT', '@USER_03ec6c70', 'RT', '@USER_90c91e77', ':', 'Vid', '&', 'footage', 'of', 'Daytona', 'Commish', 'getting', 'busted', 'for', 'Ballot', 'fraud', '.', 'http://is.gd/gmK3Y'], ["i've", 'always', 'had', 'my', 'eye', 'on', 'this', 'one', 'particular', 'guy', '....', 'i', 'was', 'too', 'shy', ',', 'so', 'i', 'decided', 'to', 'write'], ['@USER_78921f4f', 'thanks'], ['How', 'to', 'Create', 'the', 'Perfect', 'Mudroom', '(', '

In [44]:
def read_trans_prob_file(file):
    output_dic = {}
    # Output1: dictionary of tag: prob of transitioning to all other tags
    with open(file, encoding = 'utf-8') as curr:
        for line in curr.readlines():
            tag_ij, sep, trans_prob = line.strip().partition(':')
            tag_i, sep, tag_j = tag_ij.partition(' ')
            if tag_i not in output_dic.keys():
                output_dic[tag_i] = {}
                output_dic[tag_i][tag_j.strip()] = trans_prob.strip()
            else:
                output_dic[tag_i][tag_j.strip()] = trans_prob.strip()
    return output_dic

trans_prob_dic = read_trans_prob_file("trans_probs.txt")

output_prob_dic = open_file_into_dic("output_probs.txt")

print(output_prob_dic)


{'RT': {'~': '0.013426156141223273', 'N': '0.00011292417141889222', 'V': '0.00039003733214464814'}, '@USER_44285fcc': {'@': '0.00012227181023415052'}, ':': {'~': '0.011685728493286921', ',': '0.005910029837043838', 'E': '0.00044455734789787885', 'G': '0.0001905608842025027'}, 'I': {'O': '0.015995189416716776', '$': '0.00012655024044545685', 'E': '0.00012701638511367967'}, 'see': {'V': '0.0012258316153117513'}, 'u': {'O': '0.003427540589296452'}, 'Bron': {'^': '0.00012080212611741966'}, '...': {',': '0.005508377323846684', '~': '0.003853804077573347'}, '31': {'$': '0.00018982536066818528'}, 'right': {'R': '0.0007977417771232204', '!': '0.00018753516284303307', 'A': '0.0004886988393402566', 'N': '0.00011292417141889222'}, 'quick': {'A': '0.00012217470983506414'}, '@USER_a887f5eb': {'@': '0.00012227181023415052'}, 'am': {'V': '0.0007243550454114893', 'N': '0.0001693862571283383'}, 'secure': {'A': '0.00012217470983506414', 'V': '0.00011143923775561376'}, '.': {',': '0.02685333945375258', '

In [41]:
trans_prob_dic

{'START': {'~': '0.1434798005938598',
  '@': '0.30329528635121267',
  'V': '0.07900878077129134',
  '^': '0.06629618531332009',
  '#': '0.01816993107957177',
  'L': '0.03633078173381641',
  '!': '0.05903184505162223',
  'N': '0.05721575998619775',
  ',': '0.01453776094872284',
  'D': '0.029974484004830784',
  'O': '0.0726524830423057',
  'A': '0.03542273920110418',
  'G': '0.00727342068702498',
  'S': '0.002733208023463819',
  'P': '0.019077973612284003',
  'E': '0.008181463219737213',
  'R': '0.030882526537543017',
  '$': '0.00727342068702498',
  'X': '0.0018251654907515866',
  'U': '0.002733208023463819',
  '&': '0.002733208023463819',
  'M': '0.0009171229580393546',
  'Z': '0.0009171229580393546',
  'T': '9.080425327122322e-06',
  'Y': '9.080425327122322e-06'},
 '~': {'@': '0.47303421932827994',
  'O': '0.055435707641459026',
  'U': '0.07522710629960219',
  '~': '0.03366516911750154',
  '#': '0.02179032992261563',
  'X': '0.005957210996101094',
  'STOP': '0.04949828804401608',
  'A'

In [47]:
train_tags_lst = open_file_into_lst('twitter_tags.txt')

In [56]:
tweets_no_tag_lst

[['@USER_0240cb3a',
  'Yeah',
  'Im',
  'goodie',
  'just',
  'in',
  'the',
  'lab',
  'working',
  '...',
  'Wow',
  '!',
  'I',
  'need',
  'to',
  'be',
  'where',
  'you',
  'at',
  '!',
  'lol',
  'let',
  'me',
  'visit'],
 ['Does',
  'it',
  'feel',
  'like',
  'Summer',
  'to',
  'you',
  '?',
  'It',
  'does',
  'in',
  'Myrtle',
  'Beach',
  ',',
  'South',
  'Carolina',
  'its',
  '85',
  'degrees'],
 ['@USER_341450c3',
  'no',
  'one',
  'cares',
  '.',
  'ppl',
  'are',
  'used',
  'to',
  'foolish',
  'grabs',
  'for',
  'attention',
  '.',
  "they'll",
  'just',
  '*',
  'shrug',
  '*'],
 ['Red', 'wine', 'in', 'beautiful', 'china', '....'],
 ['RT',
  '@USER_f5eeff1e',
  ':',
  'RT',
  '@USER_03ec6c70',
  'RT',
  '@USER_90c91e77',
  ':',
  'Vid',
  '&',
  'footage',
  'of',
  'Daytona',
  'Commish',
  'getting',
  'busted',
  'for',
  'Ballot',
  'fraud',
  '.',
  'http://is.gd/gmK3Y'],
 ["i've",
  'always',
  'had',
  'my',
  'eye',
  'on',
  'this',
  'one',
  'particu

In [60]:
def get_max_key(dic):
    keys_lst = list(dic.keys())
    values_lst = list(dic.values())
    return keys_lst[values_lst.index(max(values_lst))]
    

def find_max_prob(lst, in_trans_dic, in_output_dic, train_tags_lst):
    num_states = len(train_tags_lst)
    # Base Case Step
    for tweet in lst: # tweet: list containing the tokens
        prob_matrix = []
        tweet_len = len(tweet) 
        x1_token = tweet[0]
        for i in range(0, tweet_len):
            prob_matrix.append({})
        #bp_matrix = []
        ######################################################################################
        # BASE CASE: Iterate through all the possible states
        # For each state, determine a0,v -> transition from START -> state v
        # determine bv(x1) -> output probability of state v -> x1
            # if (x1 & tag) was present in train data: get the output prob
                    # if  x1 was present in train data but tag was not: output prob = 0
            # else: x1 is unseen and we assign a tag V to it and its output_prob
        ######################################################################################
        for tag in train_tags_lst:
            trans_prob = in_trans_dic['START'][tag] #a0,v
            if x1_token in in_output_dic.keys():
                curr_tag = tag
                if tag in in_output_dic[x1_token].keys(): #means tag is also associated with x1_token
                    output_prob = in_output_dic[x1_token][tag] #bv(x1)
                else:
                    output_prob = 0
            else:
                output_prob = 0.15171351559491722
                curr_tag = 'V'
            total_prob = float(trans_prob) * float(output_prob)
            prob_matrix[0][curr_tag] = total_prob
        #print(prob_matrix)

        # Initialisation Step
        for index in range(1, tweet_len):
            for tag in train_tags_lst:
                trans_prob = 
                prob_matrix[index]
            
    
    
find_max_prob(tweets_no_tag_lst, trans_prob_dic, output_prob_dic, train_tags_lst)


[{'V': 0.0027566241221991827}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}]
[{'@': 0.0, ',': 0.0, 'L': 0.0, '~': 0.0, '&': 0.0, 'S': 0.0, 'N': 0.0, 'A': 0.0, 'G': 0.0, '$': 0.0, 'V': 8.8046783051531e-06, 'R': 0.0, 'X': 0.0, 'E': 0.0, 'T': 0.0, 'M': 0.0, 'D': 0.0, 'O': 0.0, 'Z': 0.0, '!': 0.0, '^': 0.0, 'U': 0.0, 'P': 0.0, 'Y': 0.0, '#': 0.0}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}]
[{'V': 0.0027566241221991827}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}]
[{'V': 0.0027566241221991827}, {}, {}, {}, {}, {}]
[{'@': 0.0, ',': 0.0, 'L': 0.0, '~': 0.0019263822058847415, '&': 0.0, 'S': 0.0, 'N': 6.461042288543589e-06, 'A': 0.0, 'G': 0.0, '$': 0.0, 'V': 3.081637406803585e-05, 'R': 0.0, 'X': 0.0, 'E': 0.0, 'T': 0.0, 'M': 0.0, 'D': 0.0, 'O': 0.0, 'Z': 0.0, '!': 0.0, '^': 0.0, 'U': 0.0, 'P': 0.0, 'Y': 0.0, '#': 0.0}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {},

In [39]:
def viterbi_predict(in_tags_filename, in_trans_probs_filename, in_output_probs_filename, in_test_filename,
                    out_predictions_filename):
    lst_lst = [[]]
    
    # Read twitter_dev_no_tag into a list of list
    tweets_no_tag_lst = file_to_lst(in_test_filename)
    
    # Read trans_probs.txt and convert to a dict
    trans_prob_dic = read_trans_prob_file(in_trans_probs_filename)
    # Read output_probs.txt and convert to a dict
    # For every token in training data, it has a dictionary of tags it is associated with
    output_prob_dic = open_file_into_dic(in_output_probs_filename)
    print(output_prob_dic)
    
    # Read in the train_tags.txt
    train_tags_lst = open_file_into_lst(in_tags_filename)
    # For each token in the tweet, calculate the product of emission and transition probs
    # use another function here
    
    
    

viterbi_predict('twitter_tags.txt', 'trans_probs.txt', 'output_probs.txt', 'twitter_dev_no_tag.txt'
               ,'viterbi_predictions.txt')

{'RT': {'~': '0.013426156141223273', 'N': '0.00011292417141889222', 'V': '0.00039003733214464814'}, '@USER_44285fcc': {'@': '0.00012227181023415052'}, ':': {'~': '0.011685728493286921', ',': '0.005910029837043838', 'E': '0.00044455734789787885', 'G': '0.0001905608842025027'}, 'I': {'O': '0.015995189416716776', '$': '0.00012655024044545685', 'E': '0.00012701638511367967'}, 'see': {'V': '0.0012258316153117513'}, 'u': {'O': '0.003427540589296452'}, 'Bron': {'^': '0.00012080212611741966'}, '...': {',': '0.005508377323846684', '~': '0.003853804077573347'}, '31': {'$': '0.00018982536066818528'}, 'right': {'R': '0.0007977417771232204', '!': '0.00018753516284303307', 'A': '0.0004886988393402566', 'N': '0.00011292417141889222'}, 'quick': {'A': '0.00012217470983506414'}, '@USER_a887f5eb': {'@': '0.00012227181023415052'}, 'am': {'V': '0.0007243550454114893', 'N': '0.0001693862571283383'}, 'secure': {'A': '0.00012217470983506414', 'V': '0.00011143923775561376'}, '.': {',': '0.02685333945375258', '

In [21]:
def naive_predict(in_output_probs_filename, in_test_filename, out_prediction_filename):
    output_lst = []
    # Read in the tokens from in_output_probs_filename into a dictionary
    token_tag_prob_dic = open_file_into_dic(in_output_probs_filename)
    
    # Read in the tokens from in_test_filename into a list
    test_tokens_lst = open_file_into_lst(in_test_filename)
    
    # For each test_token, check if it is present in the token_tag_prob_dic
    # IF PRESENT: get the highest probability tag to it
    # ELSE: Assign tag 'Y' which is the lowest likelihood tag since unseen token means it 
    #       a rare occurrence hence should be a low appearance tag
    for token in test_tokens_lst:
        if token in token_tag_prob_dic.keys():
            max_tag = get_max_key(token_tag_prob_dic[token])
            output_lst.append(max_tag)
        else:
            if token[0:5] == "@USER":
                output_lst.append("@")
            elif token[0:4] == "http":
                output_lst.append("U")
            elif token[0] == "#":
                output_lst.append("#")
            else:
                output_lst.append("Y")
    # write the output_lst to the out_prediction_file
    write_lst_to_file(output_lst, out_prediction_filename)

def naive_predict2(in_output_probs_filename, in_train_filename, in_test_filename, out_prediction_filename):
    output_lst = []
    # Read in the tokens from in_output_probs_filename into a dictionary
    token_tag_prob_dic = open_file_into_dic(in_output_probs_filename)
    
    # Read in the tokens from in_test_filename into a list
    test_tokens_lst = open_file_into_lst(in_test_filename)
    
    # Adjust the output_probabilities by multiplying by the [count(y = j) + sigma(num_words + 1)]
    naive2_token_tag_output_dic = adjust_prob_naive2(in_train_filename, token_tag_prob_dic)
    
    # For each test_token, check if it is present in the token_tag_prob_dic
    # IF PRESENT: get the highest probability tag to it
    # ELSE: Assign tag 'Y' which is the lowest likelihood tag since unseen token means it 
    #       a rare occurrence hence should be a low appearance tag
    for token in test_tokens_lst:
        if token in naive2_token_tag_output_dic.keys():
            max_tag = get_max_key(naive2_token_tag_output_dic[token])
            output_lst.append(max_tag)
        else:
            # using information from twitter_train.txt to predict tags for certain tokens
            if token[0:5] == "@USER":
                output_lst.append("@")
            elif token[0:4] == "http":
                output_lst.append("U")
            elif token[0] == "#":
                output_lst.append("#")
            else:
                output_lst.append("V")
            
    # write the output_lst to the out_prediction_file
    write_lst_to_file(output_lst, out_prediction_filename)

# def viterbi_predict(in_tags_filename, in_trans_probs_filename, in_output_probs_filename, in_test_filename,
#                     out_predictions_filename):
#     pass

# def viterbi_predict2(in_tags_filename, in_trans_probs_filename, in_output_probs_filename, in_test_filename,
#                      out_predictions_filename):
#     pass

# def forward_backward(in_train_filename, in_tag_filename, out_trans_filename, out_output_filename,
#                      max_iter, seed, thresh):
#     pass

# def cat_predict(in_test_filename, in_trans_probs_filename, in_output_probs_filename, in_states_filename,
#                 out_predictions_file):
#     pass


def evaluate(in_prediction_filename, in_answer_filename):
    """Do not change this method"""
    with open(in_prediction_filename) as fin:
        predicted_tags = [l.strip() for l in fin.readlines() if len(l.strip()) != 0]

    with open(in_answer_filename) as fin:
        ground_truth_tags = [l.strip() for l in fin.readlines() if len(l.strip()) != 0]

    assert len(predicted_tags) == len(ground_truth_tags)
    correct = 0
    for pred, truth in zip(predicted_tags, ground_truth_tags):
        if pred == truth: correct += 1
    return correct, len(predicted_tags), correct/len(predicted_tags)

def evaluate_ave_squared_error(in_prediction_filename, in_answer_filename):
    """Do not change this method"""
    with open(in_prediction_filename) as fin:
        predicted_tags = [l.strip() for l in fin.readlines() if len(l.strip()) != 0]

    with open(in_answer_filename) as fin:
        ground_truth_tags = [l.strip() for l in fin.readlines() if len(l.strip()) != 0]

    assert len(predicted_tags) == len(ground_truth_tags)
    error = 0.0
    for pred, truth in zip(predicted_tags, ground_truth_tags):
        error += (int(pred) - int(truth))**2
    return error/len(predicted_tags), error, len(predicted_tags)

def run():
    '''
    You should not have to change the code in this method. We will use it to execute and evaluate your code.
    You can of course comment out the parts that are not relevant to the task that you are working on, but make sure to
    uncomment them later.
    This sequence of code corresponds to the sequence of questions in your project handout.
    '''

    ddir = 'C:/Users/nicol/OneDrive/Desktop/Y3S1/BT3102/Project/projectfiles'

    in_train_filename = f'{ddir}/twitter_train.txt'

    naive_output_probs_filename = f'{ddir}/naive_output_probs.txt'

    in_test_filename = f'{ddir}/twitter_dev_no_tag.txt'
    in_ans_filename  = f'{ddir}/twitter_dev_ans.txt'
    naive_prediction_filename = f'{ddir}/naive_predictions.txt'
    naive_predict(naive_output_probs_filename, in_test_filename, naive_prediction_filename)
    correct, total, acc = evaluate(naive_prediction_filename, in_ans_filename)
    print(f'Naive prediction accuracy:     {correct}/{total} = {acc}')

    naive_prediction_filename2 = f'{ddir}/naive_predictions2.txt'
    naive_predict2(naive_output_probs_filename, in_train_filename, in_test_filename, naive_prediction_filename2)
    correct, total, acc = evaluate(naive_prediction_filename2, in_ans_filename)
    print(f'Naive prediction2 accuracy:    {correct}/{total} = {acc}')

#     trans_probs_filename =  f'{ddir}/trans_probs.txt'
#     output_probs_filename = f'{ddir}/output_probs.txt'

#     in_tags_filename = f'{ddir}/twitter_tags.txt'
#     viterbi_predictions_filename = f'{ddir}/viterbi_predictions.txt'
#     viterbi_predict(in_tags_filename, trans_probs_filename, output_probs_filename, in_test_filename,
#                     viterbi_predictions_filename)
#     correct, total, acc = evaluate(viterbi_predictions_filename, in_ans_filename)
#     print(f'Viterbi prediction accuracy:   {correct}/{total} = {acc}')

#     trans_probs_filename2 =  f'{ddir}/trans_probs2.txt'
#     output_probs_filename2 = f'{ddir}/output_probs2.txt'

#     viterbi_predictions_filename2 = f'{ddir}/viterbi_predictions2.txt'
#     viterbi_predict2(in_tags_filename, trans_probs_filename2, output_probs_filename2, in_test_filename,
#                      viterbi_predictions_filename2)
#     correct, total, acc = evaluate(viterbi_predictions_filename2, in_ans_filename)
#     print(f'Viterbi2 prediction accuracy:  {correct}/{total} = {acc}')

#     in_train_filename   = f'{ddir}/twitter_train_no_tag.txt'
#     in_tag_filename     = f'{ddir}/twitter_tags.txt'
#     out_trans_filename  = f'{ddir}/trans_probs4.txt'
#     out_output_filename = f'{ddir}/output_probs4.txt'
#     max_iter = 10
#     seed     = 8
#     thresh   = 1e-4
#     forward_backward(in_train_filename, in_tag_filename, out_trans_filename, out_output_filename,
#                      max_iter, seed, thresh)

#     trans_probs_filename3 =  f'{ddir}/trans_probs3.txt'
#     output_probs_filename3 = f'{ddir}/output_probs3.txt'
#     viterbi_predictions_filename3 = f'{ddir}/fb_predictions3.txt'
#     viterbi_predict2(in_tags_filename, trans_probs_filename3, output_probs_filename3, in_test_filename,
#                      viterbi_predictions_filename3)
#     correct, total, acc = evaluate(viterbi_predictions_filename3, in_ans_filename)
#     print(f'iter 0 prediction accuracy:    {correct}/{total} = {acc}')

#     trans_probs_filename4 =  f'{ddir}/trans_probs4.txt'
#     output_probs_filename4 = f'{ddir}/output_probs4.txt'
#     viterbi_predictions_filename4 = f'{ddir}/fb_predictions4.txt'
#     viterbi_predict2(in_tags_filename, trans_probs_filename4, output_probs_filename4, in_test_filename,
#                      viterbi_predictions_filename4)
#     correct, total, acc = evaluate(viterbi_predictions_filename4, in_ans_filename)
#     print(f'iter 10 prediction accuracy:   {correct}/{total} = {acc}')

#     in_train_filename   = f'{ddir}/cat_price_changes_train.txt'
#     in_tag_filename     = f'{ddir}/cat_states.txt'
#     out_trans_filename  = f'{ddir}/cat_trans_probs.txt'
#     out_output_filename = f'{ddir}/cat_output_probs.txt'
#     max_iter = 1000000
#     seed     = 8
#     thresh   = 1e-4
#     forward_backward(in_train_filename, in_tag_filename, out_trans_filename, out_output_filename,
#                      max_iter, seed, thresh)

#     in_test_filename         = f'{ddir}/cat_price_changes_dev.txt'
#     in_trans_probs_filename  = f'{ddir}/cat_trans_probs.txt'
#     in_output_probs_filename = f'{ddir}/cat_output_probs.txt'
#     in_states_filename       = f'{ddir}/cat_states.txt'
#     predictions_filename     = f'{ddir}/cat_predictions.txt'
#     cat_predict(in_test_filename, in_trans_probs_filename, in_output_probs_filename, in_states_filename,
#                 predictions_filename)

#     in_ans_filename     = f'{ddir}/cat_price_changes_dev_ans.txt'
#     ave_sq_err, sq_err, num_ex = evaluate_ave_squared_error(predictions_filename, in_ans_filename)
#     print(f'average squared error for {num_ex} examples: {ave_sq_err}')

if __name__ == '__main__':
    run()

Naive prediction accuracy:     1004/1378 = 0.7285921625544267
Naive prediction2 accuracy:    1051/1378 = 0.762699564586357
